# GraphGuard

***Locate and find Classes in Apks with updated Obfuscation Mapping***


Processing Steps:
1. String Strategy (Classes + Methods) \
  Strategy using 2 different variants only involving Strings:
  * String Counter: used in Classes and Methods and try to find exact matching counter.
  * Unique Strings: Match Classes by identifying Strings used only in this single Class.



2. Structure Strategy (Classes) \
  Strategy that enumerates all classes and tries to find an optimal match by using the following (weighted) criteria:
  * Modifiers of class
  * Modifiers, Parameters, Parameter Types and Return Types of Methods
  * Number and Types of Fields.



3. Method Strategy (Methods) \
  Strategy that uses already matched classes and tries to find an optimal Method Match in the class by using the following (weighted) criteria:
  * Modifiers
  * Return Type, Parameter Types
  * Bytecode Length
  * References to and from


4. Field Strategy (Fields) \
  Strategy that uses already matched classes and tries to find an optimal Field Match in the class by using the following (weighted criteria)
  * By Type if a matching Type has been found
  * By weighted sub-criteria such as:
    * Access Flags
    * Size
    * Number of reads & Number of writes (references)

***If you use Akrolyb, please have a look at [Akrolyb Interoptability](#Interoptability-with-Akrolyb)***

In [1]:
%matplotlib notebook
%load_ext autoreload

from IPython.core.display import display, HTML
display(HTML("<style>div.output_area pre {white-space: pre;}</style>"))

In [2]:
%autoreload
from collections.abc import Iterable

from core.start import process_files
from utils.formats import *

from core.accumulator import Accumulator
from core.decs import *

from strategies import\
    strings as strings_strategy,\
    methods as methods_strategy,\
    structures as structures_strategy,\
    fields as fields_strategy

from core.strategy_handler import StrategyHandler, FLAG_CLASS, FLAG_METHOD, FLAG_FIELD

from utils import generate
from utils import io_akrolyb

# Loading Androguard

The following code loads the files and starts Androguard

It should support multiprocessing, however the Pipe communication seems to break when transmitting the processed Androguard Objects. I suspect the Object is simply too big for Pickle to serialize or another component in the transmitting chain.

In [3]:
AG_SESSION_FILE = "./Androguard.ag"
MULTIPROCESS_FILES = False  # Currently not working due to serialization issues

apk_dir = "../../../Downloads/"

# APK Files to load
file_paths = (
    "com.snapchat.android_11.3.5.66-2114_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
    "com.snapchat.android_11.4.1.64-2116_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk"
)


# file_paths = (
#     "com.instagram.android_165.1.0.29.119-253447811_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
#     "com.instagram.android_165.1.0.29.119-253447809_minAPI19(armeabi-v7a)(420,400,360,480dpi)_apkmirror.com.apk"
# )
file_paths = tuple(map(lambda apk_name: apk_dir + apk_name, file_paths))

In [4]:
(a, d, dx), (a2, d2, dx2) = process_files(*file_paths, MULTIPROCESS_FILES)

Loading Session from Apk at ../../../Downloads/com.snapchat.android_11.3.5.66-2114_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk
Loading Session from Apk at ../../../Downloads/com.snapchat.android_11.4.1.64-2116_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk


# List of Classes, Methods and Fields

Defining Elements that GraphGuard should try to match (obviously requires full class names).

GraphGuard has full Akrolyb support. Read about it at [Akrolyb Interoptability](#Interoptability-with-Akrolyb). Using this is optional. In case you don't want to use this functionality, do not run the few last cells that are related to Akrolyb Support and Automation


* For using Akrolyb Support:
    * Use CodeGen in Akrolyb to generate a Python-compatible tuple of declarations that you can copy paste into `named_c_decs`, `named_f_decs` and `named_f_decs`.
    * Specify the correct file for auto replacing values, the "declarations files".
    * Types of the variables:
        * `named_c_decs` in form of a `Dict[str, str]`
        * `named_m_decs` in form of a `Dict[str, MethodDec]`
        * `named_f_decs` in form of a `Dict[str, FieldDec]`
    * GraphGuard extracts the declarations and uses the given names for Auto-Updates.


* For not using Akrolyb, and just simply try and match the values (without Auto-Replacing):
    * Leave named_c_decs, named_m_decs, named_f_decs empty.
    * Assign the declarations directly to c_decs, m_decs, f_decs further below by uncommenting the existing cell.
    * Types of the variables:
        * `c_decs` in form of a `Tuple[str]`
        * `m_decs` in form of a `Tuple[MethodDec]`
        * `f_decs` in form of a `Tuple[FieldDec]`

Note that classes of methods and fields are automatically merged with `c_decs`.

In [23]:
named_c_decs = {
    'SCREENSHOT_DETECTOR': 'x76',
    'STORY_VIEWED_PLUGIN': 'pgi',
    'SNAP_MODEL': 'qLc',
    'APP_START_EXPERIMENT_MANAGER': 'NCf',
    'CBC_ENCRYPTION_ALGORITHM': 'hi7',
    'CAPTION_EDIT_TEXT_VIEW': 'com.snap.previewtools.caption.ui.CaptionEditTextView'
}

In [24]:
named_m_decs = {
    'SCREENSHOT_DETECTED': MethodDec('x76', 'a', 'x76', 'w76'),
    'MARK_STORY_AS_VIEWED': MethodDec('pgi', 'X', 'fPe', 'cFe'),
    'SNAP_MODEL_CONSTRUCTOR': MethodDec('qLc', '<init>', 'java.lang.String', 'boolean', 'java.lang.String', 'java.lang.String', 'java.lang.String', 'java.lang.Long', 'gW5', 'long', 'boolean', 'java.lang.Long', 'long'),
    'FORCE_APP_DECK': MethodDec('NCf', 'f', 'tn5'),
    'DECRYPT_MEDIA_STREAM': MethodDec('hi7', 'D0', 'java.io.InputStream')
}

In [25]:
named_f_decs = {
}

In [26]:
m_decs = tuple(named_m_decs.values())
f_decs = []
for f_dec in named_f_decs.values():
    if isinstance(f_dec, Iterable):
        f_decs.extend(f_dec)
    else:
        f_decs.append(f_dec)
f_decs = tuple(f_decs)

# Add Classes of Methods and Fields to c_decs
c_decs = tuple(map(lambda m: m.class_name, m_decs)) \
        + tuple(map(lambda v: v.class_name, f_decs)) + tuple(named_c_decs.values())

In [27]:
# Without using Akrolyb's Auto-Replacing Values:

# c_decs = tuple()
# m_decs = tuple()
# f_decs = tuple()

# Processing and Matching

Loading the accumulator, an object that manages all the possible candidates that are matched by the different Matchers, and extracts the matching candidates. It also performs Inner joins on previous candidates to find the exact (or optimal) match.

In [28]:
accumulator = Accumulator()

Resolving the previously defined MethodDecs. If this fails, the MethodDecs are wrong and contain an error. Make sure the method specified with the MethodDec exists.

In [29]:
dec_cas = resolve_classes(dx, c_decs)

r_cas = tuple(dec_cas.values())
r_mas = resolve_methods(m_decs, dec_cas)
r_fas = resolve_fields(f_decs, dec_cas)

print("Resolved all Classes, Methods and Fields")
if False:
    print("", *map(pretty_format_ma, r_mas), sep="\n* ")

# Arguments to provide to Strategies
s_args = (dx, dx2, r_cas, r_mas, r_fas, accumulator)

Resolved all Classes, Methods and Fields


## Strategies & StrategyHandler

Strategies are essentially functions returning candidates for Classes, Methods or Fields. A `strategy` gets registered in the `StrategyHandler` with Flags indicating whether it tries to match Classes, Methods or Fields. If a search is invoked for a matching flag, the strategy gets invoked on the given arguments.

Global Parameters and Rules of the Strategies allow for easily changing criteria, e. g. use a more lenient search and not require strict matches.

In [30]:
strategies = StrategyHandler()


strings_strategy.MAX_USAGE_COUNT_STR = 20
strings_strategy.UNIQUE_STRINGS_MAJORITY = 2 / 3

methods_strategy.MIN_MATCH_POINTS = 2

## String Strategy

### Exact Counter Match

Extracts Strings used either in the given methods directly or in the classes the methods define for both, the old version and the new version. It then compares the Counters for classes and methods and tries to find exact matches between the Counter Objects.

In [31]:
def string_counter_strategy(r_cas, r_mas, r_fas):
    string_s = strings_strategy.StringStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs, candidates_ms = string_s.compare_counters()
    
    accumulator.add_candidates(candidates_cs, candidates_ms)

strategies.add_strategy(string_counter_strategy, FLAG_CLASS | FLAG_METHOD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

* Found multiple candidates for Method hi7#D0(java.io.InputStream)
+ Found single candidate for Method. Considering it a match 
	x76#a(x76, w76) -> ma6#a(ma6, la6)
+ Found single candidate for Method. Considering it a match 
	pgi#X(fPe, cFe) -> jsi#<init>(hsi, GFl, nge, GFl, iMi)
+ Matching Class of single candidate method match 
	Lx76; -> Lma6;
+ Matching Class of single candidate method match 
	Lpgi; -> Ljsi;
Could resolve 2 new Classes, 2 new Methods, 0 new Fields.


### Unique Strings

Gather all Strings that are used only in a single class ("Unique Strings") that we still need to match. Then try to find the matching class by only searching for the Unique Strings.

In [32]:
def unique_strings_strategy(r_cas, r_mas, r_fas):
    string_s = strings_strategy.StringStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs = string_s.compare_unique_strings()
    
    accumulator.add_candidates(candidates_cs)

strategies.add_strategy(unique_strings_strategy, FLAG_CLASS)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

+ Found single candidate for Class. Considering it a match! 
	LNCf; -> LINf;
+ Found single candidate for Class. Considering it a match! 
	Lhi7; -> Lcm7;
Could resolve 2 new Classes, 0 new Methods, 0 new Fields.


## Structure Strategy

Iterating through every single class and checks for each unmatched class if both have a similar "Profile":
* Number of Methods and Fields
* Types of Fields and Descriptors of Methods

In [33]:
def structure_strategy(r_cas, r_mas, r_fas):
    structure_s = structures_strategy.StructureStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs = structure_s.get_exact_structure_matches()

    accumulator.add_candidates(candidates_cs)

strategies.add_strategy(structure_strategy, FLAG_CLASS)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

+ Found single candidate for Class. Considering it a match! 
	LqLc; -> LHVc;
+ Found single candidate for Class. Considering it a match! 
	Lcom/snap/previewtools/caption/ui/CaptionEditTextView; -> Lcom/snap/previewtools/caption/ui/CaptionEditTextView;
Could resolve 2 new Classes, 0 new Methods, 0 new Fields.


## Method Strategy

Uses different weighted criteria to get exact or optimal matches. The criteria are:
* Modifiers
* Return Type and Parameter Types
* Length of Bytecode
* References to and from

In [34]:
def method_strategy(r_cas, r_mas, r_fas):
    method_s = methods_strategy.MethodStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)

    print("Exact Matching")
    print()

    # Exact Matches
    candidates_ms = method_s.try_resolve_ms(True)
    accumulator.add_candidates(candidates_ms=candidates_ms)

    print()
    print("Non-Exact Matching")
    print()

    method_s.update_matched()
    # Non-Exact Matches by using weights on the criteria
    candidates_ms = method_s.try_resolve_ms(False)
    accumulator.add_candidates(candidates_ms=candidates_ms)
    
strategies.add_strategy(method_strategy, FLAG_METHOD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

Exact Matching

+ Found single candidate for Method. Considering it a match 
	qLc#<init>(java.lang.String, boolean, java.lang.String, java.lang.String, java.lang.String, java.lang.Long, gW5, long, boolean, java.lang.Long, long) -> HVc#<init>(java.lang.String, boolean, java.lang.String, java.lang.String, java.lang.String, java.lang.Long, NY5, long, boolean, java.lang.Long, long)
* Found multiple candidates for Method hi7#D0(java.io.InputStream)
.. Inner join narrowed down search. ((2 | 17) -> 2)
Could resolve 0 new Classes, 1 new Methods, 0 new Fields.

Non-Exact Matching

+ Found single candidate for Method. Considering it a match 
	NCf#f(tn5) -> INf#f(Rp5)
* Found multiple candidates for Method hi7#D0(java.io.InputStream)
Could resolve 0 new Classes, 1 new Methods, 0 new Fields.


## Field Strategy

Gathers all fields of matching Class and filters using the following criteria:
* By Type if a matching Type has been found
* By weighted sub-criteria such as:
  * Access Flags
  * Size
  * Number of reads & Number of writes
* By Index if the list of Fields has not changed dramatically

In [35]:
def field_strategy(r_cas, r_mas, r_fas):
    field_s = fields_strategy.FieldStrategy(*s_args)
    
    print("Resolving Types of Fields")
    strategies.invoke_strategies(tuple(field_s.get_types_to_match()))
    print()
    
    candidates_fs = field_s.try_resolve_fs()
    
    accumulator.add_candidates(candidates_fs=candidates_fs)
    
strategies.add_strategy(field_strategy, FLAG_FIELD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

# Results
Display summary and matching pairs.

Output to compatible MethodDec input to update from auto-updated values.

In [36]:
print(len(accumulator.matching_cs), "/", len(c_decs), "Classes were matched")
print(len(accumulator.matching_ms), "/", len(m_decs), "Methods were matched")
print(len(accumulator.matching_fs), "/", len(f_decs), "Fields were matched")

print()
print("Matching Classes:")
for c1, c2 in accumulator.matching_cs.items():
    print("•", pretty_format_class(c1), "->", pretty_format_class(c2))

print()
print("Matching Methods:")
for m, ma in accumulator.matching_ms.items():
    print("•", pretty_format_ma(m), "->", pretty_format_ma(ma))

print()
print("Matching Fields:")
for fa, fa2 in accumulator.matching_fs.items():
    print("•", pretty_format_fa(fa), "->", pretty_format_fa(fa2))

6 / 11 Classes were matched
4 / 5 Methods were matched
0 / 0 Fields were matched

Matching Classes:
• x76 -> ma6
• pgi -> jsi
• NCf -> INf
• hi7 -> cm7
• qLc -> HVc
• com.snap.previewtools.caption.ui.CaptionEditTextView -> com.snap.previewtools.caption.ui.CaptionEditTextView

Matching Methods:
• x76#a(x76, w76) -> ma6#a(ma6, la6)
• pgi#X(fPe, cFe) -> jsi#<init>(hsi, GFl, nge, GFl, iMi)
• qLc#<init>(java.lang.String, boolean, java.lang.String, java.lang.String, java.lang.String, java.lang.Long, gW5, long, boolean, java.lang.Long, long) -> HVc#<init>(java.lang.String, boolean, java.lang.String, java.lang.String, java.lang.String, java.lang.Long, NY5, long, boolean, java.lang.Long, long)
• NCf#f(tn5) -> INf#f(Rp5)

Matching Fields:


In [37]:
generate.generate_m_decs(m_decs, r_mas, accumulator.matching_ms)
generate.generate_c_decs(c_decs, r_cas, accumulator.matching_cs)
generate.generate_f_decs(f_decs, r_fas, accumulator.matching_fs)

m_decs = (
    MethodDec('ma6', 'a', 'ma6', 'la6'),
    MethodDec('jsi', '<init>', 'hsi', 'GFl', 'nge', 'GFl', 'iMi'),
    MethodDec('HVc', '<init>', 'java.lang.String', 'boolean', 'java.lang.String', 'java.lang.String', 'java.lang.String', 'java.lang.Long', 'NY5', 'long', 'boolean', 'java.lang.Long', 'long'),
    MethodDec('INf', 'f', 'Rp5'),
    # No match for Method hi7#D0(java.io.InputStream),
)
c_decs = (
    'ma6'
    'jsi'
    'HVc'
    'INf'
    'cm7'
    'ma6'
    'jsi'
    'HVc'
    'INf'
    'cm7'
    'com.snap.previewtools.caption.ui.CaptionEditTextView',
)
f_decs = (
)


# Warnings

Shows some warnings that result in common bugs, so that the Hooks (or other kind of modifications applied) must be updated logic-wise. 

Currently supports: 
* Change in number of parameters 

In [42]:
for m_old, m_new in accumulator.matching_ms.items():
    def get_number_of_params(m):
        return len(tuple(get_pretty_params(str(m.get_descriptor()))))
    
    if get_number_of_params(m_old) != get_number_of_params(m_new):
        print("Warning: Number of parameters changed for:", pretty_format_ma(m_old), "-", pretty_format_ma(m_new))

# Interoptability with Akrolyb

GraphGuard was build with Akrolyb in mind and has full Akrolyb support, meaning that it is not only capable of automatically matching the given declarations, but also automatically editing the "declaration file" to update matched declarations and mark unmatched items with a `/* TODO */` comment.

The following Code snippet allows to "extract" MethodDecs from MemberDeclarations in Akrolyb. This automates the process of providing GraphGuard the hooks it should find. See [this](#List-of-Classes,-Methods-and-Fields) to learn how to use GraphGuard's Akrolyb Support.

## Extract

It is not in Python, since it would require a Kotlin Parser and evaluating imports, variables etc. Just executing the Constructors in Kotlin, then getting the values is much easier than static analysis. 

The `main` function can (and should) be run statically (locally on the machine and not on your Android) to build the list of `MethodDec`s that GraphGuard is supposed to match in an updated build of the target application. Adjust the MemberDeclarations Class to the Class where you declared the `MemberDeclaration`s.


```kotlin
fun main() {
    GraphGuard.printGeneratedDecs(ClassDecs, MemberDecs, VariableDecs)
}
```

In [39]:
file_dir = "/home/jaqxues/CodeProjects/IdeaProjects/SnipTools/"
c_file = file_dir + "packimpl/src/main/java/com/jaqxues/sniptools/packimpl/hookdec/ClassDeclarations.kt"
m_file = file_dir + "packimpl/src/main/java/com/jaqxues/sniptools/packimpl/hookdec/MemberDeclarations.kt"
f_file = "/dev/null"


# file_dir = "/home/jaqxues/CodeProjects/IdeaProjects/Instaprefs/"
# c_file = file_dir + "packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/ClassDecsV8.kt"
# m_file = file_dir + "packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/MemberDecsV8.kt"
# f_file = file_dir + "packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/VariableDecsV8.kt"

In [40]:
%autoreload
c_txt = io_akrolyb.replace_cs(c_file, accumulator)
m_txt = io_akrolyb.replace_ms(m_file, accumulator, named_m_decs)
f_txt = io_akrolyb.replace_fs(f_file, accumulator, named_f_decs)

No matching Method found for hi7#D0(java.io.InputStream)


In [41]:
print(c_txt)
print(m_txt)
print(f_txt)

package com.jaqxues.sniptools.packimpl.hookdec

import com.jaqxues.akrolyb.genhook.decs.ClassDec

object ClassDeclarations {
    val SCREENSHOT_DETECTOR = ClassDec("ma6")
    val STORY_VIEWED_PLUGIN = ClassDec("jsi")
    val SNAP_MODEL = ClassDec("HVc")
    val APP_START_EXPERIMENT_MANAGER = ClassDec("INf")
    val CBC_ENCRYPTION_ALGORITHM = ClassDec("cm7")
    val CAPTION_EDIT_TEXT_VIEW = ClassDec("com.snap.previewtools.caption.ui.CaptionEditTextView")
}
package com.jaqxues.sniptools.packimpl.hookdec

import com.jaqxues.akrolyb.genhook.decs.MemberDec.ConstructorDec
import com.jaqxues.akrolyb.genhook.decs.MemberDec.MethodDec
import com.jaqxues.sniptools.packimpl.features.*
import com.jaqxues.sniptools.packimpl.hookdec.ClassDeclarations.APP_START_EXPERIMENT_MANAGER
import com.jaqxues.sniptools.packimpl.hookdec.ClassDeclarations.CBC_ENCRYPTION_ALGORITHM
import com.jaqxues.sniptools.packimpl.hookdec.ClassDeclarations.SCREENSHOT_DETECTOR
import com.jaqxues.sniptools.packimpl.hookdec.ClassD